<a href="https://colab.research.google.com/github/gmauricio-toledo/NLP-LCC/blob/main/Rest-Mex/Extender_vectores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Más información: https://sites.google.com/cimat.mx/rest-mex-2025/home

In [11]:
import re
import nltk
from nltk import word_tokenize
from nltk import word_tokenize
from string import punctuation

nltk.download('punkt_tab')
nltk.download('stopwords')

stopwords_list = nltk.corpus.stopwords.words('spanish')
punct_set = set(punctuation)

def clean_document(doc, joint=True):
    # Convertir a minúsculas
    doc = doc.lower()
    # Eliminar etiquetas HTML números y símbolos
    doc = re.sub(r'<[^>]+>', ' ', doc)
    doc = re.sub(r'\d+', ' ', doc)
    doc = re.sub(r'[^a-z\s]', ' ', doc)

    tokens = word_tokenize(doc, language='english')
    # Filtrar stopwords y puntuación
    cleaned_tokens = [
        token for token in tokens
        if token not in stopwords_list
        and token not in punct_set
        and len(token) > 1
    ]

    if joint:
        return ' '.join(cleaned_tokens)
    else:
        return cleaned_tokens


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import pandas as pd

url = "https://raw.githubusercontent.com/gmauricio-toledo/NLP-LCC/main/Rest-Mex/Rest-Mex_2025_train.csv"
df = pd.read_csv(url)
df.head()

,Title,Review,Polarity,Town,Region,Type
0,Mi Lugar Favorito!!!!,Excelente lugar para comer y pasar una buena n...,5.0,Sayulita,Nayarit,Restaurant
1,lugares interesantes para visitar,"andar mucho, así que un poco difícil para pers...",4.0,Tulum,QuintanaRoo,Attractive
2,No es el mismo Dreams,"Es nuestra cuarta visita a Dreams Tulum, elegi...",3.0,Tulum,QuintanaRoo,Hotel
3,un buen panorama cerca de CancÃºn,"Estando en CancÃºn, fuimos al puerto y tomamos...",4.0,Isla_Mujeres,QuintanaRoo,Attractive
4,El mejor,Es un lugar antiguo y por eso me encanto tiene...,5.0,Patzcuaro,Michoacan,Hotel


In [2]:
!gdown 11ThQG0qBax3AOyx23jiexERV5ympt_qJ

Downloading...
From: https://drive.google.com/uc?id=11ThQG0qBax3AOyx23jiexERV5ympt_qJ
To: /content/lemma_db.csv
100% 55.9M/55.9M [00:00<00:00, 95.1MB/s]


In [4]:
import pandas as pd

df = pd.read_csv('lemma_db.csv')
df

,Review,Town
0,lugar favorito excelente lugar comer pasar bue...,Sayulita
1,lugar interesante visitar andar asi dificil pe...,Tulum
2,mismo dreams cuarto visita dreams tulum eleg...,Tulum
3,buen panorama cerca cancaon cancaon puerto tom...,Isla_Mujeres
4,mejor lugar antiguo encantar area juego gigant...,Patzcuaro
...,...,...
208046,excelente ambiente comida atencion excelente r...,Tequisquiapan
208047,mal servicio alli huesped dia playa camarero t...,Tulum
208048,excelente excelente comida asi atencion servic...,Ixtapan_de_la_Sal
208049,visita nocturno tocar visitar sitio habia caid...,Creel


#1. Polaridad

In [7]:
df1 = df[['Title','Review','Region','Polarity']]
df1

,Title,Review,Region,Polarity
0,Mi Lugar Favorito!!!!,Excelente lugar para comer y pasar una buena n...,Nayarit,5.0
1,lugares interesantes para visitar,"andar mucho, así que un poco difícil para pers...",QuintanaRoo,4.0
2,No es el mismo Dreams,"Es nuestra cuarta visita a Dreams Tulum, elegi...",QuintanaRoo,3.0
3,un buen panorama cerca de CancÃºn,"Estando en CancÃºn, fuimos al puerto y tomamos...",QuintanaRoo,4.0
4,El mejor,Es un lugar antiguo y por eso me encanto tiene...,Michoacan,5.0
...,...,...,...,...
208046,"excelente ambiente, comida y atencion","Excelente Restaurante, comida Mexicana de alto...",Queretaro,5.0
208047,Muy mal servicio,Estuvimos allí como huéspedes de un día a la p...,QuintanaRoo,2.0
208048,Excelente,"Excelente comida, así como la atención y servi...",Estado_de_Mexico,5.0
208049,Visita nocturna,Nos toco visitar este sitio cuando ya había ca...,Chihuahua,4.0


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


df1['Text'] = df1['Title'] + ' ' + df1['Review']
df1.dropna(subset=['Text'], inplace=True)

X = df1['Text'].values

y = df1['Polarity']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y.astype(int),
                                                    test_size=0.2,
                                                    random_state=2025)

train_docs = [clean_document(doc) for doc in X_train]
test_docs = [clean_document(doc) for doc in X_test]

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(train_docs)
X_test_tfidf = vectorizer.transform(test_docs)

<ipython-input-12-cf0b8a4b4df4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Text'] = df1['Title'] + ' ' + df1['Review']
<ipython-input-12-cf0b8a4b4df4>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.dropna(subset=['Text'], inplace=True)


In [13]:
train_docs[:5]

['mas peque as mar pie imperdible cuenta conseguimos guia opac experiencia lugar gico ls ruinas mas peque as borde playa fabulosas gran lugar combinar visitas playa paraiso cenote dos ojos',
 'san panchito nunca visto multitud devotos peregrinos aquellos asisten fiesta anual san francisco as san panchito iglesia hermosamente decorado ocasi sica ma ana hab peregrinos ciudad fiesta',
 'atendieron servicio simo dieron servicio ten an grupo grande literal batearon turistas',
 'cumplea pintoresca cerca mar preciosas vistas alojamos nivel calle habitaciones gustado ofrecido especial sorpresa mitad cumplea llegamos sugerir mantener hermosa embargo puertas madera plegables payne triple tormenta puertas vidrio evitar ruido calle fuerte sentir seguro mantener aire acondicionado mejor cajero autom tico hotel hu spedes conseguir pesos restaurante oferta hu spedes cada lugar nuevo lugar especial hace sentir ninguna sugerencias personal agradable servicial',
 'gran lugar aqu propia ten problemas qu 

In [20]:
import numpy as np

vocab = vectorizer.get_feature_names_out()
important_words = []

for label in np.unique(y_train):
    X_j = X_train_tfidf[y_train==label].copy()
    normas = np.sum(np.abs(X_j), axis=0)
    word_normas = [(word, norma) for word, norma in zip(vocab, normas.tolist()[0])]
    word_normas = sorted(word_normas, key=lambda x: x[1], reverse=True)[:5]
    print(f"Label: {label}")
    for word, norma in word_normas:
        print(word,end=' ')
    print()
    important_words.extend([word for word, norma in word_normas])

Label: 0
servicio mal simo hotel mala 
Label: 1
servicio comida hotel mal lugar 
Label: 2
comida lugar est hotel bien 
Label: 3
lugar buena comida buen hotel 
Label: 4
lugar excelente comida servicio mejor 


In [24]:
important_words = list(set(important_words))
print(important_words)
important_words_idxs = [vocab.tolist().index(word) for word in important_words]
print(important_words_idxs)
extra_features = X_train_tfidf[:, important_words_idxs].toarray()
extra_features.shape

['buena', 'mala', 'bien', 'mejor', 'servicio', 'mal', 'lugar', 'est', 'hotel', 'buen', 'excelente', 'comida', 'simo']
[12591, 53062, 10996, 54962, 78447, 53061, 52417, 34216, 43696, 12590, 35287, 19027, 79162]


(166439, 13)

In [17]:
!pip install -qq gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 17.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [19]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

documents = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(train_docs)]

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

# 2. Tipo de lugar

In [ ]:
df2 = df[['Title','Review','Region','Type']]
df2

,Title,Review,Region,Type
0,Mi Lugar Favorito!!!!,Excelente lugar para comer y pasar una buena n...,Nayarit,Restaurant
1,lugares interesantes para visitar,"andar mucho, así que un poco difícil para pers...",QuintanaRoo,Attractive
2,No es el mismo Dreams,"Es nuestra cuarta visita a Dreams Tulum, elegi...",QuintanaRoo,Hotel
3,un buen panorama cerca de CancÃºn,"Estando en CancÃºn, fuimos al puerto y tomamos...",QuintanaRoo,Attractive
4,El mejor,Es un lugar antiguo y por eso me encanto tiene...,Michoacan,Hotel
...,...,...,...,...
208046,"excelente ambiente, comida y atencion","Excelente Restaurante, comida Mexicana de alto...",Queretaro,Restaurant
208047,Muy mal servicio,Estuvimos allí como huéspedes de un día a la p...,QuintanaRoo,Hotel
208048,Excelente,"Excelente comida, así como la atención y servi...",Estado_de_Mexico,Restaurant
208049,Visita nocturna,Nos toco visitar este sitio cuando ya había ca...,Chihuahua,Attractive


#3. Pueblo

In [ ]:
df3 = df[['Title','Review','Region','Town']]
df3

,Title,Review,Region,Town
0,Mi Lugar Favorito!!!!,Excelente lugar para comer y pasar una buena n...,Nayarit,Sayulita
1,lugares interesantes para visitar,"andar mucho, así que un poco difícil para pers...",QuintanaRoo,Tulum
2,No es el mismo Dreams,"Es nuestra cuarta visita a Dreams Tulum, elegi...",QuintanaRoo,Tulum
3,un buen panorama cerca de CancÃºn,"Estando en CancÃºn, fuimos al puerto y tomamos...",QuintanaRoo,Isla_Mujeres
4,El mejor,Es un lugar antiguo y por eso me encanto tiene...,Michoacan,Patzcuaro
...,...,...,...,...
208046,"excelente ambiente, comida y atencion","Excelente Restaurante, comida Mexicana de alto...",Queretaro,Tequisquiapan
208047,Muy mal servicio,Estuvimos allí como huéspedes de un día a la p...,QuintanaRoo,Tulum
208048,Excelente,"Excelente comida, así como la atención y servi...",Estado_de_Mexico,Ixtapan_de_la_Sal
208049,Visita nocturna,Nos toco visitar este sitio cuando ya había ca...,Chihuahua,Creel


# Test

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/gmauricio-toledo/NLP-LCC/main/Rest-Mex/Rest-Mex_2025_test.xlsx"
df = pd.read_excel(url)
df.drop(columns='ID',inplace=True)
df.head()

,Title,Review
0,Una de las mejores comidas en Sayulita,"Comida exelente, el lugar muy cómodo y persona..."
1,Me gustó mucho...,En mi camino hacia Mahahual paré a comer en es...
2,Excelentes restaurantes en hotel Guaycura,"Excelente Servicio, la comida es deliciosa tan..."
3,Gran desayuno y almuerzo,Mi esposa y yo fuimos allí dos veces. Una vez ...
4,Mejor atención de la zona hotelera!,"Increíble restaurant! La comida riquísima, la ..."
